## Homework 1 CS524
Lorenzo Lu

## 1. Warmup

a).<br>
I will choose Clp solver. Though Clp is the most specialized solver among this three, the very fast speed is the main reason for my choice.

b).

In [1]:
using JuMP, Clp
m = Model();

@variable(m, 0 <= x1 <= 3);
@variable(m, 0 <= x2 <= 3);
@variable(m, 0 <= x3 <= 3);
@constraint(m, 2*x1 <= x2 - x3 + 2);
@objective(m, Max, 6*x1 - 2*x2 + 3*x3);

@time optimize!(m, with_optimizer(Clp.Optimizer));

println(termination_status(m));
println("Build x1:", JuMP.value(x1));
println("Build x2:", JuMP.value(x2));
println("Build x3:", JuMP.value(x3));
println("Total ", JuMP.objective_value(m));


 12.039782 seconds (52.21 M allocations: 2.582 GiB, 7.66% gc time)
OPTIMAL
Build x1:1.0
Build x2:3.0
Build x3:3.0
Total 9.0
Coin0506I Presolve 1 (0) rows, 3 (0) columns and 3 (0) elements
Clp0006I 0  Obj 0 Dual inf 8.9999998 (2)
Clp0006I 1  Obj 9
Clp0000I Optimal - objective value 9
Clp0032I Optimal objective 9 - 1 iterations time 0.002


b). <br>
The optimal objective value is 9.0.The optimal x1 is 1 and x2 is 3.0

c).<br>
I changed the coefficient 2 at the constraint into -10, rendering this problem infeasible.

In [2]:
m = Model();

@variable(m, 0 <= x1 <= 3);
@variable(m, 0 <= x2 <= 3);
@variable(m, 0 <= x3 <= 3);
@constraint(m, 2*x1 <= x2 - x3 - 10);
@objective(m, Max, 6*x1 - 2*x2 + 3*x3);

@time optimize!(m, with_optimizer(Clp.Optimizer));

println(termination_status(m));
println("Build x1:", JuMP.value(x1));
println("Build x2:", JuMP.value(x2));
println("Build x3:", JuMP.value(x3));
println("Total ", JuMP.objective_value(m));

  0.000727 seconds (1.79 k allocations: 129.875 KiB)
INFEASIBLE
Coin0507I Presolve determined that the problem was infeasible with tolerance of 1e-08
Clp3003W Analysis indicates model infeasible or unbounded
Clp0006I 0  Obj 0 Primal inf 9.9999999 (1) Dual inf 8.9999998 (2)
Clp0006I 2  Obj -27 Primal inf 6.9999999 (1)
Clp0006I 2  Obj -27 Primal inf 6.9999999 (1)
Clp0001I Primal infeasible - objective value -27
Clp0032I PrimalInfeasible objective -27 - 2 iterations time 0.002


ErrorException: Primal solution not available

## 2. Stigler diet

a).<br>
The decision variables are the amount of each kind of food. In mathematical model, they are $n_i$, where i stands for different kinds of food.<br>
The objective is to minimize the cost of the food. Minimumize $\sum_i^nn_i*1$, as each row could be bought by 1 dollar.<br>
The constraints are that each nutrition should meet the minimum daily requirement. ${\forall}_jf_{i,j}n_{i,j} \geq F_{min,j}$, where i is a certain kind of food, where j is a certain kind of nutrition.

b).

In [3]:
# STARTER CODE TO LOAD DATA FOR STIGLER'S DIET PROBLEM

# Add packages
using Pkg;
#Pkg.add("CSV");

using CSV;

# import Stigler's data set as a dataframe
raw = CSV.read("stigler.csv");

(m,n) = size(raw);  # m = number of rows, n = number of columns

n_nutrients = 2:n;      # indices of columns containing nutrients (skip the first one)
n_foods = 2:m;          # indices of rows containing food names (skip the two first ones)

nutrients = names(raw)[n_nutrients];   # the list of nutrients 
foods = raw[n_foods,1];                # the list of foods 

# Put the data about nutrients and foods into an array
data = convert(Matrix{Float64}, raw[n_foods,n_nutrients]);

# lower[i] is the minimum daily requirement of nutrient i.
lower = convert(Vector{Float64}, raw[1,n_nutrients]);

In [4]:
stigler = Model();

n_food = @variable(stigler, [1:m-1]);
for i = 1 : m-1
    @constraint(stigler,n_food[i] >= 0);
end

for j = 1:n-1
    @constraint(stigler,n_food' * data[:,j]>=lower[j] );
end

@objective(stigler,Min,sum(n_food));
@time optimize!(stigler, with_optimizer(Clp.Optimizer));

  1.580117 seconds (5.50 M allocations: 284.096 MiB, 3.45% gc time)
Coin0506I Presolve 9 (-77) rows, 76 (-1) columns and 569 (-78) elements
Clp0006I 0  Obj 0 Primal inf 5.1310537 (9)
Clp0006I 6  Obj 0.10866228
Clp0000I Optimal - objective value 0.10866228
Coin0511I After Postsolve, objective 0.10866228, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0.1086622782 - 6 iterations time 0.002, Presolve 0.00


In [5]:
println(termination_status(stigler));
println("Optimal daily cost = ", JuMP.objective_value(stigler)," dollars");
println("Optimal annual cost = ", 365 * JuMP.objective_value(stigler)," dollars");

OPTIMAL
Optimal daily cost = 0.10866227820675685 dollars
Optimal annual cost = 39.66173154546625 dollars


c).<br>
Yes, the optimal has been calculated.This result is a little bit smaller than Stigler's. The composition of the foods and their amounts are listed below:

In [6]:
println(foods,JuMP.value.(n_food));

["Wheat Flour (Enriched)", "Macaroni", "Wheat Cereal (Enriched)", "Corn Flakes", "Corn Meal", "Hominy Grits", "Rice", "Rolled Oats", "White Bread (Enriched)", "Whole Wheat Bread", "Rye Bread", "Pound Cake", "Soda Crackers", "Milk", "Evaporated Milk (can)", "Butter", "Oleomargarine", "Eggs", "Cheese (Cheddar)", "Cream", "Peanut Butter", "Mayonnaise", "Crisco", "Lard", "Sirloin Steak", "Round Steak", "Rib Roast", "Chuck Roast", "Plate", "Liver (Beef)", "Leg of Lamb", "Lamb Chops (Rib)", "Pork Chops", "Pork Loin Roast", "Bacon", "Ham, smoked", "Salt Pork", "Roasting Chicken", "Veal Cutlets", "Salmon, Pink (can)", "Apples", "Bananas", "Lemons", "Oranges", "Green Beans", "Cabbage", "Carrots", "Celery", "Lettuce", "Onions", "Potatoes", "Spinach", "Sweet Potatoes", "Peaches (can)", "Pears (can)", "Pineapple (can)", "Asparagus (can)", "Green Beans (can)", "Pork and Beans (can)", "Corn (can)", "Peas (can)", "Tomatoes (can)", "Tomato Soup (can)", "Peaches, Dried", "Prunes, Dried", "Raisins, Drie

d).

In [7]:
raw = CSV.read("stigler_vege.csv");

(m,n) = size(raw);  # m = number of rows, n = number of columns

n_nutrients = 2:n;      # indices of columns containing nutrients (skip the first one)
n_foods = 2:m;          # indices of rows containing food names (skip the two first ones)

nutrients = names(raw)[n_nutrients];   # the list of nutrients 
foods = raw[n_foods,1];                # the list of foods 

# Put the data about nutrients and foods into an array
data = convert(Matrix{Float64}, raw[n_foods,n_nutrients]);

# lower[i] is the minimum daily requirement of nutrient i.
lower = convert(Vector{Float64}, raw[1,n_nutrients]);

stigler_vege = Model();

n_food = @variable(stigler_vege, [1:m-1]);
for i = 1 : m-1
    @constraint(stigler_vege,n_food[i] >= 0);
end

for j = 1:n-1
    @constraint(stigler_vege,n_food' * data[:,j]>=lower[j] );
end

@objective(stigler_vege,Min,sum(n_food));
@time optimize!(stigler_vege, with_optimizer(Clp.Optimizer));



  0.001371 seconds (6.17 k allocations: 393.672 KiB)
Coin0506I Presolve 9 (-53) rows, 52 (-1) columns and 393 (-54) elements
Clp0006I 0  Obj 0 Primal inf 4.5508622 (9)
Clp0006I 5  Obj 0.10904446
Clp0000I Optimal - objective value 0.10904446
Coin0511I After Postsolve, objective 0.10904446, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0.1090444587 - 5 iterations time 0.002, Presolve 0.00


In [8]:
println(termination_status(stigler_vege));
println("Optimal daily cost = ", JuMP.objective_value(stigler_vege)," dollars");
println("Optimal annual cost = ", 365 * JuMP.objective_value(stigler_vege)," dollars");

OPTIMAL
Optimal daily cost = 0.10904445867706583 dollars
Optimal annual cost = 39.80122741712903 dollars


The annual cost is higher than that in c). It is as expected.

##  3. Polyhedron modeling

a).<br>
The matrix A should be $ \left ( \begin{matrix} 1 & 0 & 0 \\ 0 & 1 & 0 \\ 0 & 0 & 1 \\ -1 & 0 & 0 \\ 0 & -1 & 0 \\ 0 & 0 & -1 \end{matrix}\right ) $, and the vector b is  $ \left ( \begin{matrix} 1  \\ 1 \\ 1 \\ 1 \\ 1 \\1 \end{matrix}\right ) $.

b).<br>
The matrix A should be $ \left ( \begin{matrix} 1 & 1 & 1 \\ -1& 1 & 1 \\ 1& -1& 1\\ 1 & 1 &-1 \\ 1& -1 & -1 \\ -1& 1& -1\\ -1 & -1 & 1 \\ -1 & -1 & -1 \end{matrix}\right ) $, and the vector b is  $ \left ( \begin{matrix}1 \\ 1 \\ 1\\1 \\ 1 \\ 1\\ 1 \\1  \end{matrix}\right ) $.

## 4. Standard form with equality constraints

a).<br> $\mathbf{x} = \begin{pmatrix} x_1 \\ x_2 \\ x_3 \\ x_4 \\ x_5 \\ x_6 \\ x_7 \\ x_8 \end{pmatrix} = \begin{pmatrix} 3 - z_1 + 6z_2 - z_3 + z_4 \\ 2 - (z_3 + z_4) \\ z_2 + 1 \\ 5 - z_2 \\ z_3 + 1 \\ 5 - z_3 \\ z_4 + 2\\ 2 - z4 \end{pmatrix}$ <br>
$\mathbf{A} = \begin{pmatrix} 0 & 0 & 1 & 1 & 0 & 0 & 0 & 0 \\ 0 & 0 & 0 & 0 & 1 & 1 & 0 & 0 \\ 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 \\ 0 & 1 & 0 & 0 & 1 & 0 & 1 & 0 \\0 & 0 & 1 & 0 & 0 & 0 & 1 & 0 \end{pmatrix}$ <br>
$\mathbf{b} = \begin{pmatrix} 6\\6\\4\\5\\0 \end{pmatrix}$ <br>
$\mathbf{c} = \begin{pmatrix} -3\\0\\-17\\0\\-3\\0\\3 \\0\end{pmatrix}$

b).

In [9]:

using JuMP, Clp
ieqm = Model();
@variable(ieqm, z1);
@variable(ieqm, -1 <= z2 <= 5);
@variable(ieqm, -1 <= z3 <= 5);
@variable(ieqm, -2 <= z4 <= 2);
@constraint(ieqm, -z1 + 6z2 - z3 + z4 >= -3);

@constraint(ieqm, 7z2 + z4 == 5);
@constraint(ieqm, z3 + z4 <= 2);

@objective(ieqm, Max, 3z1 - z2);

@time optimize!(ieqm, with_optimizer(Clp.Optimizer));

println(termination_status(ieqm));
println("Build z1:", JuMP.value(z1));
println("Build z2:", JuMP.value(z2));
println("Build z3:", JuMP.value(z3));
println("Build z4:", JuMP.value(z4));
println("Total ", JuMP.objective_value(ieqm));

  1.129688 seconds (3.28 M allocations: 167.633 MiB, 2.09% gc time)
OPTIMAL
Build z1:8.571428571428571
Build z2:0.42857142857142855
Build z3:-1.0
Build z4:2.0
Total 25.28571428571429
Coin0506I Presolve 0 (-3) rows, 0 (-4) columns and 0 (-8) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 25.285714
Coin0511I After Postsolve, objective 25.285714, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 25.28571429 - 0 iterations time 0.002, Presolve 0.00


In [10]:
using JuMP, Clp
eqm = Model();
@variable(eqm,x1>=0);
@variable(eqm,x2>=0);
@variable(eqm,x3>=0);
@variable(eqm,x4>=0);
@variable(eqm,x5>=0);
@variable(eqm,x6>=0);
@variable(eqm,x7>=0);
@variable(eqm,x8>=0);

@constraint(eqm, x3 + x4 == 6);
@constraint(eqm, x5 + x6 == 6);
@constraint(eqm, x7 + x8 == 4);
@constraint(eqm, x2 + (x5) + (x7) == 5);
@constraint(eqm, -7*(x3) + (x7) == 0);




@objective(eqm,Max,(-3)x1 + (-17)x3 + (-3)x5 + (3)x7  + 23);
@time optimize!(eqm, with_optimizer(Clp.Optimizer));

x1_val = JuMP.value(x1);
x3_val = JuMP.value(x3);
x5_val = JuMP.value(x5);
x7_val = JuMP.value(x7);

println(termination_status(eqm));
println("Build z1:", 3 - x1_val + 6(1-x3_val) -(x5_val-1) + (x7_val-2));
println("Build z2:", 1 - x3_val);
println("Build z3:", x5_val - 1);
println("Build z4:", x7_val - 2);
println("Total ", JuMP.objective_value(eqm));

  0.054573 seconds (155.00 k allocations: 7.780 MiB, 14.33% gc time)
OPTIMAL
Build z1:8.571428571428571
Build z2:0.4285714285714286
Build z3:-1.0
Build z4:2.0
Total 25.285714285714285
Coin0506I Presolve 0 (-5) rows, 0 (-8) columns and 0 (-11) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 25.285714
Coin0511I After Postsolve, objective 25.285714, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 25.28571429 - 0 iterations time 0.002, Presolve 0.00
